In [ ]:
# https://www.kaggle.com/code/sarmat/lgbm-stacking-example/notebook

In [1]:
import numpy as np # linear algebra
import pandas as pd
import random
from scipy.sparse.linalg import svds

from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import torch
from tqdm import tqdm
from dataset import custom_train_test_split, make_dataset

from sklearn.metrics import RocCurveDisplay, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc


def get_metric(targets, preds):
    auc = roc_auc_score(targets, preds)
    acc = accuracy_score(targets, np.where(preds >= 0.5, 1, 0))
    precsion = precision_score(targets, np.where(preds >= 0.5, 1, 0))
    recall = recall_score(targets, np.where(preds >= 0.5, 1, 0))
    F1_score = f1_score(targets, np.where(preds >= 0.5, 1, 0))

    print('auc :',auc)
    print('acc :',acc)
    print('precision :',precsion)
    print('recall :',recall)

def test_to_csv(preds, name:str):
    
    result = []
    for n,i in enumerate(preds):
        row = {}    
        row['id'] = n
        row['prediction'] = i
        result.append(row)
    pd.DataFrame(result).to_csv(f'output/{name}.csv', index=None)
    


In [2]:
cate_cols = [
            # 'assessmentItemID',
            # 'testId',
            # 'KnowledgeTag',
            'hour',
            'dow',
            'i_head',
            'i_mid',
            'i_tail',
]
cont_cols = [                        
            'user_correct_answer',
            'user_total_answer',
            'user_acc',            
            't_elapsed',            
            'cum_correct',
            'last_problem',
            'head_term',
            # 'left_asymptote',
            'elo_prob',
            'pkt',
            'u_head_mean',
            'u_head_count',
            'u_head_std',
            'u_head_elapsed',
            'i_mid_elapsed',
            'i_mid_mean',
            'i_mid_std',
            'i_mid_sum',
            'i_mid_count',
            'i_mid_tag_count',
            'assessment_mean',
            'assessment_sum',
            # 'assessment_std',
            'tag_mean',
            'tag_sum',
            # 'tag_std',
            'tail_mean',
            'tail_sum',
            # 'tail_std',
            'hour_mean',
            'hour_sum',
            # 'hour_std',
            'dow_mean',
            'dow_sum',
            # 'dow_std',
            'tag_elapsed',
            'tag_elapsed_o',
            'tag_elapsed_x',
            'assessment_elapsed',
            'assessment_elapsed_o',
            'assessment_elapsed_x',
            'tail_elapsed',
            'tail_elapsed_o',
            'tail_elapsed_x'
            ]

FEATS = cate_cols + cont_cols

In [3]:
data = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/all.pkl')
label = data.answerCode.to_numpy()

In [4]:
model01 = LogisticRegression(max_iter=500, C= 1)
model02 = LogisticRegression(max_iter=500, C= 2)
model03 = LogisticRegression(max_iter=500, C= 3)
model04 = LogisticRegression(max_iter=500, C= 4)
model05 = LogisticRegression(max_iter=500, C= 5)
model06 = LogisticRegression(max_iter=500, C= 6)
model07 = LogisticRegression(max_iter=500, C= 7)
model08 = LogisticRegression(max_iter=500, C= 8)
model09 = LogisticRegression(max_iter=500, C= 9)
model10 = LogisticRegression(max_iter=500, C= 10)

In [5]:
valid_user = pd.read_csv('/opt/ml/input/data/cv_valid_data.csv').userID.unique()
X_train = data[data.userID.isin(valid_user)==False]
y_train = X_train.answerCode.to_numpy()
X_valid = data[data.userID.isin(valid_user)==True]
y_valid = X_valid.answerCode.to_numpy()

In [6]:
model01.fit(X_train[FEATS], y_train)
model02.fit(X_train[FEATS], y_train)
model03.fit(X_train[FEATS], y_train)
model04.fit(X_train[FEATS], y_train)
model05.fit(X_train[FEATS], y_train)
model06.fit(X_train[FEATS], y_train)
model07.fit(X_train[FEATS], y_train)
model08.fit(X_train[FEATS], y_train)
model09.fit(X_train[FEATS], y_train)
model10.fit(X_train[FEATS], y_train)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

LogisticRegression(C=10, max_iter=500)

In [24]:
valid_predict01 = model01.predict_proba(X_valid[FEATS])[:,-1]
valid_predict02 = model02.predict_proba(X_valid[FEATS])[:,-1]
valid_predict03 = model03.predict_proba(X_valid[FEATS])[:,-1]
valid_predict04 = model04.predict_proba(X_valid[FEATS])[:,-1]
valid_predict05 = model05.predict_proba(X_valid[FEATS])[:,-1]
valid_predict06 = model06.predict_proba(X_valid[FEATS])[:,-1]
valid_predict07 = model07.predict_proba(X_valid[FEATS])[:,-1]
valid_predict08 = model08.predict_proba(X_valid[FEATS])[:,-1]
valid_predict09 = model09.predict_proba(X_valid[FEATS])[:,-1]
valid_predict10 = model10.predict_proba(X_valid[FEATS])[:,-1]

# item_id2idx는 train에서 사용한 것을 다시 사용한다.
test = pd.read_pickle('/opt/ml/level2-dkt-level2-recsys-08/data_pkl/test_data-1.pkl')
test = test[test.answerCode==-1]

test_predict01 = model01.predict_proba(test[FEATS])[:,-1]
test_predict02 = model02.predict_proba(test[FEATS])[:,-1]
test_predict03 = model03.predict_proba(test[FEATS])[:,-1]
test_predict04 = model04.predict_proba(test[FEATS])[:,-1]
test_predict05 = model05.predict_proba(test[FEATS])[:,-1]
test_predict06 = model06.predict_proba(test[FEATS])[:,-1]
test_predict07 = model07.predict_proba(test[FEATS])[:,-1]
test_predict08 = model08.predict_proba(test[FEATS])[:,-1]
test_predict09 = model09.predict_proba(test[FEATS])[:,-1]
test_predict10 = model10.predict_proba(test[FEATS])[:,-1]


print("AUC LR1:{} ".format(get_metric(y_valid, np.array(valid_predict01))))
print("AUC LR2:{} ".format(get_metric(y_valid, np.array(valid_predict02))))
print("AUC LR3:{} ".format(get_metric(y_valid, np.array(valid_predict03))))
print("AUC LR4:{} ".format(get_metric(y_valid, np.array(valid_predict04))))
print("AUC LR5:{} ".format(get_metric(y_valid, np.array(valid_predict05))))
print("AUC LR6:{} ".format(get_metric(y_valid, np.array(valid_predict06))))
print("AUC LR7:{} ".format(get_metric(y_valid, np.array(valid_predict07))))
print("AUC LR8:{} ".format(get_metric(y_valid, np.array(valid_predict08))))
print("AUC LR9:{} ".format(get_metric(y_valid, np.array(valid_predict09))))
print("AUC LR0:{} ".format(get_metric(y_valid, np.array(valid_predict10))))

auc : 0.7634685545926725
acc : 0.730521972938819
precision : 0.755321754446303
recall : 0.8708402174892608
AUC LR1:None 
auc : 0.7637364584771221
acc : 0.7305140994500368
precision : 0.7538482262668967
recall : 0.8741686442969149
AUC LR2:None 
auc : 0.763537304208074
acc : 0.7303133254860895
precision : 0.7549750596174073
recall : 0.8711526330018925
AUC LR3:None 
auc : 0.7639358738617359
acc : 0.7304944157280812
precision : 0.7567740380079874
recall : 0.8675177986722341
AUC LR4:None 
auc : 0.763535739984506
acc : 0.7306991264364197
precision : 0.7554083908213668
recall : 0.8710444891705968
AUC LR5:None 
auc : 0.7649024244130227
acc : 0.7297700547601145
precision : 0.757305972112602
recall : 0.8647060590585479
AUC LR6:None 
auc : 0.7635235407558321
acc : 0.7305574036383392
precision : 0.7552747012305031
recall : 0.8710264651987143
AUC LR7:None 
auc : 0.7635758017396683
acc : 0.7304196175846498
precision : 0.7550015357304611
recall : 0.8713328727207186
AUC LR8:None 
auc : 0.7635076965432

In [28]:
new_valid = X_valid[FEATS].copy()
new_valid.loc[:,'predict01'] = valid_predict01
new_valid.loc[:,'predict02'] = valid_predict02
new_valid.loc[:,'predict03'] = valid_predict03
new_valid.loc[:,'predict04'] = valid_predict04
new_valid.loc[:,'predict05'] = valid_predict05
new_valid.loc[:,'predict06'] = valid_predict06
new_valid.loc[:,'predict07'] = valid_predict07
new_valid.loc[:,'predict08'] = valid_predict08
new_valid.loc[:,'predict09'] = valid_predict09
new_valid.loc[:,'predict10'] = valid_predict10


# valid_tail = new_valid[new_valid.index.isin(X_valid.groupby('userID').tail(1).index)==True]
# new_valid = new_valid[new_valid.index.isin(X_valid.groupby('userID').tail(1).index)==False]

new_test = test[FEATS].copy()
new_test.loc[:,'predict01'] = test_predict01
new_test.loc[:,'predict02'] = test_predict02
new_test.loc[:,'predict03'] = test_predict03
new_test.loc[:,'predict04'] = test_predict04
new_test.loc[:,'predict05'] = test_predict05
new_test.loc[:,'predict06'] = test_predict06
new_test.loc[:,'predict07'] = test_predict07
new_test.loc[:,'predict08'] = test_predict08
new_test.loc[:,'predict09'] = test_predict09
new_test.loc[:,'predict10'] = test_predict10

In [29]:
Final = LogisticRegression()
Final.fit(new_valid, y_valid)

/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
preds = Final.predict_proba(new_test)[:,-1]

In [31]:
from datetime import date, datetime, timezone, timedelta

KST = timezone(timedelta(hours=9))
time_record = datetime.now(KST)
_day = str(time_record)[:10]
_time = str(time_record.time())[:8]
now_time = _day+'_'+_time


test_to_csv(preds, f'Blending_LR_{now_time}')